<a href="https://colab.research.google.com/github/DeeeTeeee/ibm.ai_Engineering_Professional_Cert/blob/main/NeuralNetworkRegressionModelswithKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

In [100]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

In [101]:
#read in the dataset:
concrete = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Let us check to understand our data first

In [102]:
concrete.shape

(1030, 9)

we can see our dataset has 1030 rowws and 9 columns, we will now use the describe() funcion to know the statistics of our data

In [103]:
concrete.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


the `Strength` column been the target, we can see that the min is 2 and the max is 87.

Lets go on by checking if there are any null or missing values in our dataset:


In [104]:
concrete.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

its clean and no `null` value.
Let us split the dataset into predictors and target

In [105]:
#split data:
# concrete_cols = concrete.columns
# predictors = concrete[concrete_cols[concrete_cols !='Strength']]
# target = concrete['Strength']
# Split data into predictors and target
concrete_cols = concrete.columns
predictors = concrete.loc[:, concrete_cols != 'Strength']
target = concrete['Strength']

In [106]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [107]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

We can try to normalize our data now:

In [108]:
predictors_normal = (predictors - predictors.mean()) / predictors.std()
predictors_normal

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597


Lets save the column number

In [109]:
n_cols = predictors.shape[1]
n_cols

8

## Build the Neural Network

In [110]:
#from keras.src.engine.training import optimizer
#define regression model we will use:
def regression_model(n_cols):
  model = Sequential()
  model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1))

  #compile the model:
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

In [111]:
regression_model

<function __main__.regression_model(n_cols)>

## Lets Train and Test the Network

In [112]:
model = regression_model(n_cols)

In [113]:
#fit the model:
model.fit(predictors_normal, target, validation_split = 0.3, epochs= 100, verbose=2)

Epoch 1/100
23/23 - 1s - loss: 1683.7950 - val_loss: 1176.8256 - 1s/epoch - 44ms/step
Epoch 2/100
23/23 - 0s - loss: 1585.5414 - val_loss: 1091.0361 - 87ms/epoch - 4ms/step
Epoch 3/100
23/23 - 0s - loss: 1449.5839 - val_loss: 967.0419 - 127ms/epoch - 6ms/step
Epoch 4/100
23/23 - 0s - loss: 1244.4423 - val_loss: 791.4470 - 97ms/epoch - 4ms/step
Epoch 5/100
23/23 - 0s - loss: 961.4830 - val_loss: 580.9697 - 92ms/epoch - 4ms/step
Epoch 6/100
23/23 - 0s - loss: 652.5408 - val_loss: 381.7388 - 87ms/epoch - 4ms/step
Epoch 7/100
23/23 - 0s - loss: 395.5760 - val_loss: 249.9617 - 94ms/epoch - 4ms/step
Epoch 8/100
23/23 - 0s - loss: 275.4817 - val_loss: 191.0972 - 99ms/epoch - 4ms/step
Epoch 9/100
23/23 - 0s - loss: 237.1956 - val_loss: 177.9617 - 92ms/epoch - 4ms/step
Epoch 10/100
23/23 - 0s - loss: 221.6407 - val_loss: 171.2812 - 87ms/epoch - 4ms/step
Epoch 11/100
23/23 - 0s - loss: 209.2512 - val_loss: 166.7726 - 87ms/epoch - 4ms/step
Epoch 12/100
23/23 - 0s - loss: 200.6471 - val_loss: 164.